# Tutorial 2: Part A
## `Fraction`

### Author: Vedant Prakash Shenoy

`fractions` is a library in the Python Standard Library that allows us to define (no points for guessing) fractions. This allows us to calculate HCFs and LCMs, do the normal arithmetic operations, and allow for different types of formatting (eg: improper fractions <-> mixed fractions).

To illustrate how we can leverage classes, let us try to make a similar class for fractions ourselves!

At any point, if you are confused about what to do, consult the [holy archives](https://www.google.com)

### Before Attempting this Tutorial:
1. Revise what classes are (Lecture 3) and what dunder methods do (operator overloading)
2. Watch [Python Class Development Toolkit](https://www.youtube.com/watch?v=HTLu2DFOdTg)

### During this Tutorial
1. Fill in the code to do what is described in the instructions.
2. Write your own test cases.
3. Try to think of different ways to implement the features that we want our class to have

We will cover the following concepts in roughly the same order:
1. Data Attributes and Methods (including `__init__` and `__repr__`)
2. Input validation and Exception Handling
3. `@property` and `@property.setter`
4. More dunder methods: arithmetic operations, relational operatations and absolute value
5. `@classmethods` and alternate constructors

If you see something new in the above list, revise Lecture 3 and watch the video linked above.

### After this Tutorial
1. Notice how we incrementally develop the class `Fraction`, and find new test cases that break our code, and continue the process iteratively. This testing can by automated using 'Unit Testing'. Try to find out how this is done in Python; it may be useful for you in your project!
2. Use the concepts you have learnt here, and try to apply them in a field you are interested in.
3. Go watch this video to find out when you should not be using classes: [Stop Writing Classes](https://www.youtube.com/watch?v=o9pEzgHorH0)

---
To start off with, we need to implement the concept of a fraction. At its core, a fraction simply has a numerator (`num`) and a denominator (`den`); and dividing them gives the decimal value (`decimal`)

We'll also add a pretty `repr` so that we can see what the fraction is.

In [1]:
class Fraction:
    """Class to implement fractions"""
    
    def __init__(self, num, den):
        """Initialize instance using numerator and denominator"""
        self.num = num
        self.den = den
        
    def decimal(self):
        return self.num/self.den
    
    def __repr__(self):
        return f"{self.num}/{self.den} = {self.decimal()}"

In [2]:
f = Fraction(1, 2)
print(f.num, f.den, f.decimal())

1 2 0.5


In [3]:
Fraction(1, 3)

1/3 = 0.3333333333333333

In [4]:
Fraction(-1, 3)

-1/3 = -0.3333333333333333

Looks good so far. However:

In [5]:
Fraction(1.5, 1)

1.5/1 = 1.5

In [6]:
Fraction(1, -3)

1/-3 = -0.3333333333333333

In [7]:
Fraction(0, 1)

0/1 = 0.0

In [8]:
Fraction(1, 0)

ZeroDivisionError: division by zero

---
What happened in those last two cells? There are three basic properties of fractions that we haven't yet implemented: 
1. `num` and `den` are integers,
2. `den` can't be zero,
3. the sign is written next to the numerator.

Let us modify the class to reflect this as well.

In [9]:
class Fraction:
    """Class to implement fractions"""
    
    def __init__(self, num, den):
        """Initialize instance using numerator and denominator"""
        # Modify the __init__ function from above to do the checks that we wanted.
        if not (isinstance(num, int) and isinstance(den, int)):
            raise TypeError(f"`num` and `den` must both be integers, not '{type(num).__name__}' and '{type(den).__name__}'")
        if den == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        
        self.num = num
        if den < 0:
            self.num = -self.num
        self.den = abs(den)

    def decimal(self):
        return self.num/self.den    
    
    def __repr__(self):
        return f"{self.num}/{self.den} = {self.decimal()}"

In [10]:
# Expected: a TypeError (since `num` is not an integer)
Fraction(1.2, 3)

TypeError: `num` and `den` must both be integers, not 'float' and 'int'

In [11]:
# Expected: ZeroDevisionError (since `den` is zero)
Fraction(1, 0)

ZeroDivisionError: `den` must not be 0

In [12]:
# Expected: Fraction -1/2 and not 1/-2
Fraction(1, -2)

-1/2 = -0.5

---
Our problems with wrong input types and values are now solved! We can now move on with implementing operations right?

However, a new bug shows up!

In [13]:
f = Fraction(1, 2)
f.num = 3.2
f.den = 1.2

In [14]:
f

3.2/1.2 = 2.666666666666667

Python allows for attributes to be accessed and changed (there are no private or protected variables here).

This means that if the user changes the numerator and denominator later on, the checks that we did in `__init__` are no longer done.

How can we solve this? Simple, rename `num` as `_num` and so on. This is a convention that lets people know not to mess with that attribute.

But then, the value of `num` is not accessible directly. We could write a method `num()` to return the value of `self._num`?

A solution that lets us keep the earlier API (and shows that `num` is a data attribute or a property rather than a method) is to use the `@property` decorator. Complete the class definition below:

In [15]:
class Fraction:
    """Class to implement fractions"""
    
    def __init__(self, num, den):
        """Initialize instance using numerator and denominator"""
        # Copy your _init__ from above
        if not (isinstance(num, int) and isinstance(den, int)):
            raise TypeError(f"`num` and `den` must both be integers, not '{type(num).__name__}' and '{type(den).__name__}'")
        if den == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        
        self._num = num
        if den < 0:
            self._num = -self.num
        self._den = abs(den)
        
    def decimal(self):
        return self.num/self.den
    
    @property
    def num(self):
        return self._num
    
    @property
    def den(self):
        return self._den

    def __repr__(self):
        return f"{self._num}/{self._den} = {self.decimal()}"   

In [16]:
f = Fraction(1, 2)
f.num, f.den

(1, 2)

In [17]:
f.num = 2

AttributeError: can't set attribute

In [18]:
f

1/2 = 0.5

---
This solves our problem of worrying about the user breaking our code. However, keep in mind that if they want to, they still can (change `f._num` instead in the cell above)


Note how this varies somewhat from the `C++ Way of Doing Things`<sup>TM</sup>, where you might write a `get_num()` method to get the value and a `set_num()` method to set the value (if you wish to implement such a feature).

The idiomatic way to do it in Python is to use `@property` and `@property.setter`. Again, we show the process for `num`. Complete the code for `den` to make `Fraction` a mutable, consistent container for fractions.

In [19]:
class Fraction:
    """Class to implement fractions"""
    
    def __init__(self, num, den):
        """Initialize instance using numerator and denominator"""
        # Copy __init__ from above
        if not (isinstance(num, int) and isinstance(den, int)):
            raise TypeError(f"`num` and `den` must both be integers, not '{type(num).__name__}' and '{type(den).__name__}'")
        if den == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        
        self._num = num
        if den < 0:
            self._num = -self.num
        self._den = abs(den)

    def decimal(self):
        return self.num/self.den
    
    @property
    def num(self):
        return self._num
    
    @num.setter
    def num(self, value):
        if isinstance(value, int):
            self._num = value
        else:
            raise TypeError(f"`num` must be of type 'int', not '{type(value).__name__}'")
    
    # Put in @property and @property.setter for den as well
    @property
    def den(self):
        return self._den
    
    @den.setter
    def den(self, value):
        if not isinstance(value, int):
            raise TypeError(f"`den` must be of type 'int', not '{type(value).__name__}'")
        elif value == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        else:
            self._den = abs(value)
            if value < 0:
                self._num = -self._num
                
    def __repr__(self):
        return f"{self._num}/{self._den} = {self.decimal()}"

In [20]:
f = Fraction(1, 2)

In [21]:
f.num = 2
f.den = -1

In [22]:
f

-2/1 = -2.0

---
At this point, we have a class `Fraction` which does has all the checks we wanted it to have, is mutable and remains consistent at all times.

Let us take the plunge and add more features

## Feature Request
Implement the following features into the class `Fraction`
1. A method to get the simplest form of a fraction (2/4 --> 1/2)
2. Modify `Fraction.decimal()` to take an optional argument `fmt`, which if provided returns a formatted string with the given format. For example:

                                            f = Fraction(1, 3)
                                            print(f.decimal())
                                            # Out: 0.3333333333333333
                                            
                                            print(f.decimal('4.2f'))
                                            # Out: 0.33
                                            
                                            print(f.decimal('.2e')
                                            # Out: 3.33e-01
3. Define the `abs` of a `Fraction`
4. Make the `repr` a bit more expressive: include the class name. When someone prints an instance of `Fraction`, only show the `num`/`den` form. On the interactive console, this should look something like:

                                            In : f = Fraction(-2, 4)

                                            In : f
                                            Out: Fraction: -2/4 = -0.5

                                            In : print(f)
                                            -2/4

(Hint: Look up the dunder method `__str__` and how it differs from `__repr__`)

In [23]:
from math import gcd

In [24]:
class Fraction:
    """Class to implement fractions"""
    
    def __init__(self, num, den):
        """Initialize instance using numerator and denominator"""
        # You know what to do here.
        if not (isinstance(num, int) and isinstance(den, int)):
            raise TypeError(f"`num` and `den` must both be integers, not '{type(num).__name__}' and '{type(den).__name__}'")
        if den == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        
        self._num = num
        if den < 0:
            self._num = -self.num
        self._den = abs(den)

    def decimal(self, fmt=None):
        """Format the decimal output according to fmt and return a string. If not passed, return a float."""
        # You can maybe try to use `if fmt is None:`
        if fmt is None:
            return self.num/self.den
        else:
            return f"{self.num/self.den:{fmt}}"
        
    def simple(self):
        """Return a Fraction instance with num and den in simplest form"""
        factor = gcd(self.num, self.den)
        return self.__class__(self.num//factor, self.den//factor)
    @property
    def num(self):
        return self._num
    
    @num.setter
    def num(self, value):
        if isinstance(value, int):
            self._num = value
        else:
            raise TypeError(f"`num` must be of type 'int', not '{type(value).__name__}'")
    
    @property
    def den(self):
        return self._den
    
    @den.setter
    def den(self, value):
        if not isinstance(value, int):
            raise TypeError(f"`den` must be of type 'int', not '{type(value).__name__}'")
        elif value == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        else:
            self._den = abs(value)
            if value < 0:
                self._num = -self._num
                
    def __repr__(self):
        return f"{self.__class__.__name__}: {self._num}/{self._den} = {self.decimal()}"
    
    def __str__(self):
        return f"{self._num}/{self._den}"

    def __abs__(self):
        return self.__class__(abs(self.num), self.den)

In [25]:
f = Fraction(-2, 4)

In [26]:
f

Fraction: -2/4 = -0.5

In [27]:
print(f)

-2/4


In [28]:
f.simple()

Fraction: -1/2 = -0.5

In [29]:
f.decimal('0.3f')

'-0.500'

In [30]:
abs(f)

Fraction: 2/4 = 0.5

---
## Feature Request
Implement the following features into the class `Fraction`
1. The operations for addition, subtraction, multiplication, and division (as per the usual definitions for fractions). Return an instance of `Fraction` with the numerator and denominator in the simplest form. 
2. Define the relational operators (`==`, `>`, `<`, `>=`, `<=`, `!=`) for fractions.


3. Make all the above operations compatible with normal numbers (`int` and `float` data types) as well. Use the test case

                                        Fraction(4, 100) == 0.16 - 0.12

In [31]:
from math import gcd, isclose

In [32]:
# Copy the class definition from above and add __dunder__ methods to do all these things.
# If you do not implement the compatibility with `float`, 
# then raise an exception, so that the user knows not to do it.


class Fraction:
    """Class to implement fractions"""
    
    def __init__(self, num, den):
        """Initialize instance using numerator and denominator"""
        if not (isinstance(num, int) and isinstance(den, int)):
            raise TypeError(f"`num` and `den` must both be integers, not '{type(num).__name__}' and '{type(den).__name__}'")
        if den == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        
        self._num = num
        if den < 0:
            self._num = -self.num
        self._den = abs(den)
    
    def decimal(self, fmt=None):
        if fmt is None:
            return self.num/self.den
        else:
            return f"{self.num/self.den:{fmt}}"
    
    def simple(self):
        factor = gcd(self.num, self.den)
        return self.__class__(self.num//factor, self.den//factor)
    
    @property
    def num(self):
        return self._num
    
    @num.setter
    def num(self, value):
        if isinstance(value, int):
            self._num = value
        else:
            raise TypeError(f"`num` must be of type 'int', not '{type(value).__name__}'")
    
    @property
    def den(self):
        return self._den
    
    @den.setter
    def den(self, value):
        if not isinstance(value, int):
            raise TypeError(f"`den` must be of type 'int', not '{type(value).__name__}'")
        elif value == 0:
            raise ZeroDivisionError(f"`den` must not be 0")
        else:
            self._den = abs(value)
            if value < 0:
                self._num = -self._num
                
    def __repr__(self):
        return f"{self.__class__.__name__}: {self._num}/{self._den} = {self.decimal()}"
    
    def __str__(self):
        return f"{self._num}/{self._den}"

    def __abs__(self):
        return self.__class__(abs(self.num), self.den)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.num * other.den == other.num  * self.den
        elif isinstance(other, (int, float)):
            return isclose(self.decimal(), other)

    def __lt__(self, other):
        if isinstance(other, self.__class__):
            return self.num * other.den < other.num * self.den
        elif isinstance(other, (int, float)):
            return self.decimal() < other and not self == other
    
    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self.num * other.den > other.num * self.den
        elif isinstance(other, (int, float)):
            return self.decimal() > other and not self == other

    def __le__(self, other):
        if isinstance(other, self.__class__):
            return self.num * other.den <= other.num * self.den
        elif isinstance(other, (int, float)):
            return self.decimal() <= other or self == other
    
    def __ge__(self, other):
        if isinstance(other, self.__class__):
            return self.num * other.den >= other.num * self.den
        elif isinstance(other, (int, float)):
            return self.decimal() >= other or self == other
        
    def __add__(self, other):
        if isinstance(other, self.__class__):
            num = self.num * other.den + other.num * self.den
            den = self.den * other.den
            return self.__class__(num, den).simple()
        elif isinstance(other, int):
            return self.__class__(self.num + other * self.den, self.den).simple()

    def __sub__(self, other):
        if isinstance(other, self.__class__):
            num = self.num * other.den - other.num * self.den
            den = self.den * other.den
            return self.__class__(num, den).simple()
        elif isinstance(other, int):
            return self.__class__(self.num - other * self.den, self.den).simple()            

    def __mul__(self, other):
        if isinstance(other, self.__class__):
            num = self.num * other.num
            den = other.den * self.den
            return self.__class__(num, den).simple()
        elif isinstance(other, int):
            return self.__class__(self.num * other, self.den).simple()

    def __truediv__(self, other):
        if isinstance(other, self.__class__):
            num = self.num * other.den
            den = other.num * self.den
            return self.__class__(num, den).simple()
        elif isinstance(other, int):
            return self.__class__(self.num, self.den * other).simple()
    

In [33]:
f1 = Fraction(1, 2)
f2 = Fraction(5, 6)

In [34]:
f1 + f2

Fraction: 4/3 = 1.3333333333333333

---
## Feature Request
Alternative constructors (good time to remember what you saw in that YouTube video)
1. Add support for mixed fractions. Figure out how you want to implement this yourself.

2. Make alternative constructors for `Fraction`. Implement the following ways of making a `Fraction` instance:

                                Fraction.from_string("1/2")  # This should be equivalent to Fraction(1, 2)
                                Fraction.from_tuple((2))  # Fraction(2, 1)
                                Fraction.from_tuple((2, 3))  # Fraction(2, 3)
                                Fraction.from_tuple((1, 2, 3))  # Fraction(5, 3): a mixed fraction
                                Fraction.from_dict(dict(num=2, den=3))  # Fraction(2, 3)

# Concluding Notes

We hope you guys had fun writing out this class. Have a look at the `fractions` library to see how much of its functionality you have replicated in this tutorial, and how you can improve it.

On Friday afternoon (Jun 4), we will release Part B of the tutorial. Good luck!

                                                __fin__
                                                
---